In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [67]:
import os
for dirname, _, filenames in os.walk('train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [68]:
acc = pd.read_csv("Accident_Information.csv")
acc.head()

C:\Users\mkvar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,...,Police_Force,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Year,InScotland
0,200501BS00001,A,3218.0,NaN,0.0,Serious,None,2005-01-04,Tuesday,1.0,...,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,17:42,Urban,Raining no high winds,2005,No
1,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,...,Metropolitan Police,Dry,Dual carriageway,None,30.0,17:36,Urban,Fine no high winds,2005,No
2,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,...,Metropolitan Police,Dry,Single carriageway,None,30.0,00:15,Urban,Fine no high winds,2005,No
3,200501BS00004,A,3220.0,NaN,0.0,Slight,None,2005-01-07,Friday,1.0,...,Metropolitan Police,Dry,Single carriageway,None,30.0,10:35,Urban,Fine no high winds,2005,No
4,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-10,Monday,1.0,...,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,21:13,Urban,Fine no high winds,2005,No


In [69]:
veh = pd.read_csv("Vehicle_Information.csv", encoding='ISO-8859-1')
acc.shape

(2047256, 34)

In [83]:
df2['Accident_Severity'] = df2['Accident_Severity'].astype('category')

In [84]:
df2 = df2[[c for c in df2 if c not in ['Accident_Severity']] 
       + ['Accident_Severity']]

In [85]:
y = df2['Accident_Severity']
X = df2.iloc[:,:-1]

In [86]:
df2.to_csv("data.csv")

In [87]:
oe = OrdinalEncoder()
oe.fit(X)
X = oe.transform(X)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [89]:
# feat = SelectKBest(score_func=chi2, k='all')
# feat.fit(X_train, y_train)
# X_train = feat.transform(X_train)
# X_test = feat.transform(X_test)

In [90]:
from imblearn.over_sampling import SMOTE

In [91]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

In [92]:
print("Before")
print(y.value_counts())
print("After")
print(pd.Series(y_resampled).value_counts())

Before
Slight     1368445
Serious     199235
Fatal        20633
Name: Accident_Severity, dtype: int64
After
Fatal      1231884
Serious    1231884
Slight     1231884
Name: Accident_Severity, dtype: int64


In [93]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1, stratify = y_resampled)

In [94]:
X_train.shape

(2956521, 15)

# XGBoost without SMOTE

In [95]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [96]:
model = XGBClassifier(verbose=1)
model.fit(X_train, y_train)

C:\Users\mkvar\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:03:12] WARNING: ..\src\learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:03:23] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbose=1, verbosity=None)

In [97]:
# make predictions for test data
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report

In [98]:
# evaluate predictions
report=classification_report(y_test,y_pred)

print("\t\t  XGBoost Classifier\n", report)

		  XGBoost Classifier
               precision    recall  f1-score   support

       Fatal       0.78      0.84      0.81    246377
     Serious       0.80      0.59      0.68    246377
      Slight       0.84      1.00      0.91    246377

    accuracy                           0.81    739131
   macro avg       0.81      0.81      0.80    739131
weighted avg       0.81      0.81      0.80    739131



# XGBoost with SMOTE

In [112]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1, stratify = y_resampled)

In [113]:
model = XGBClassifier(verbose=1)
model.fit(X_train, y_train)

C:\Users\mkvar\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:43:25] WARNING: ..\src\learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:43:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbose=1, verbosity=None)

In [114]:
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report

In [115]:
print("\t\t  XGBoost Classifier with SMOTE\n", report)

		  XGBoost Classifier with SMOTE
               precision    recall  f1-score   support

       Fatal       0.78      0.84      0.81    197101
     Serious       0.80      0.59      0.68    197102
      Slight       0.84      1.00      0.91    197102

    accuracy                           0.81    591305
   macro avg       0.80      0.81      0.80    591305
weighted avg       0.80      0.81      0.80    591305

